In [ ]:
!pip install openai                 # OpenAI GPT
!pip install langchain==0.0.103

In [3]:
import os
from dotenv import dotenv_values

api_keys = dotenv_values('keys.txt')
os.environ['OPENAI_API_KEY'] = api_keys['OPENAI_API_KEY']

In [4]:
from langchain.llms import OpenAI

model_name = "gpt-3.5-turbo"

llm = OpenAI(model_name=model_name, temperature=0.1)

In [25]:
from typing import List, Tuple, Dict, Generator

def create_formatted_history(history_messages: List[dict]) -> List[Tuple[str, str]]:
    formatted_history = []
    user_messages = []
    assistant_messages = []

    for message in history_messages:
        if message["role"] == "user":
            user_messages.append(message["content"])
        elif message["role"] == "assistant":
            assistant_messages.append(message["content"])

        if user_messages and assistant_messages:
            formatted_history.append(
                ("".join(user_messages), "".join(assistant_messages))
            )
            user_messages = []
            assistant_messages = []

    # append any remaining messages
    if user_messages:
        formatted_history.append(("".join(user_messages), None))
    elif assistant_messages:
        formatted_history.append((None, "".join(assistant_messages)))

    return formatted_history

In [26]:
question = "When is the Devoxx event?"

history_messages = []
history_messages.append({"role": "system", "content": "You are a conference organiser which knows everything about the event. Answer always truthfully."})

def ask_question(question):
    history_messages.append({"role": "user", "content": question})

    chat_generator = llm.client.create(
        messages = history_messages, 
        stream = True, 
        model = model_name
    )

    response_message = ""
    for chunk in chat_generator:
        if "choices" in chunk:
            for choice in chunk["choices"]:
                if "delta" in choice and "content" in choice["delta"]:
                    new_token = choice["delta"]["content"]
                    # Add the latest token:
                    response_message += new_token
                    # Update the assistant's response in our model:
                    history_messages[-1]["content"] = response_message

                if "finish_reason" in choice and choice["finish_reason"] == "stop":
                    break
    formatted_history = create_formatted_history(history_messages)

    print(formatted_history)

In [27]:
ask_question("When is the next Devoxx Belgium event?")
    

[("I'm sorry, but I don't have access to the latest information regarding Devoxx Belgium's events. As an AI language model, my knowledge is limited to what I have been trained on. However, you could check their official website or social media pages for the most recent updates.", None)]
